In [ ]:
from asteroid_filterbanks import STFTFB, Encoder, Decoder
from asteroid.masknn import TDConvNet , TDConvNetpp
from asteroid_filterbanks import make_enc_dec
from asteroid import ConvTasNet
from asteroid.data import LibriMix
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid.data import LibriMix
from asteroid.engine.system import System
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid import ConvTasNet

import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd


import os
import csv
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import pandas as pd
import soundfile as sf

from torchsummary import summary
import torch.nn as nn
import torch
from torchvision import transforms, utils
import torchaudio

import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#device = 'cpu'
import torch.nn.functional as F
import random


cuda


In [ ]:
def returnNormedVector_oneSam(A = None):
    
    n_channels = 1
    n_sam = A.shape[-1]
    AA = A.clone()
    t_zro = torch.zeros(n_channels, n_sam+1)
    t_zro[:,:n_sam] = AA
    AA = t_zro
    
    #AA = AA.view(A.size(0), -1)
    AA -= AA.min(1, keepdim=True)[0]
    AA /= AA.max(1, keepdim=True)[0]
    AA = AA-AA[0][n_sam]
    
    AA = AA.view(n_channels, n_sam+1)
    
    return AA[:,:n_sam]


In [ ]:
class S2Lpairs_Dataset(Dataset):

    def __init__(self, csv_file, wav_dir,mix_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.frame_ = pd.read_csv(csv_file)
        self.wav_dir = wav_dir
        self.mix_dir = mix_dir
        self.transform = transform

    def __len__(self):
        return len(self.frame_)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        spec = ['alarm','baby','crash','dog','engine','femaleScream',
                 'femaleSpeech','fire','footsteps','knock','maleScream',
                 'maleSpeech','phone','piano']
        
        _1 =            self.frame_.iloc[idx, 1] # First ROI
        _2 =            self.frame_.iloc[idx, 2] # second ROI
        pres_abs =      self.frame_.iloc[idx, 4] # 0 or 1
        cond_look_for = self.frame_.iloc[idx, 3] # Conditional. This is what we are looking for.
        
        spec.index(cond_look_for) + 1
        cond = np.zeros(14) # <<-- No. of Classes 14 in case of NIGEN
        cond[spec.index(cond_look_for)] = 1
        cond_ = torch.from_numpy(np.float32(cond)) # This is what to look for, hence a conditional
        cond_ = cond_.unsqueeze(0)
        
        class_spec_cond_wav = random.sample(sorted(os.listdir(self.wav_dir +cond_look_for)), 1)
        
        _1_sam, sr = torchaudio.load(self.wav_dir +(_1.split('_')[3]) + '/' + _1)
        _2_sam, sr = torchaudio.load(self.wav_dir + (_2.split('_')[3]) + '/' + _2)
        #_cond_sam, sr = torchaudio.load(self.wav_dir + cond_look_for + '/' + class_spec_cond_wav[0])
        
        mix_pth = self.mix_dir + _1.split('.wav')[0] + '_' + _2
        #print(">> Trying to open mix dir:", mix_pth)
        
        mix_1_2, sr = torchaudio.load(mix_pth)
        
        #_1_sam = returnNormedVector_oneSam(A = _1_sam)
        #_2_sam = returnNormedVector_oneSam(A = _2_sam)
        
        #mix_1_2 = _1_sam + _2_sam
        #mix_1_2 = returnNormedVector_oneSam(A = mix_1_2)
        
        if (pres_abs == 1):
            if(cond_look_for in _1):
                desired_sep = _1_sam
                
            elif(cond_look_for in _2):
                desired_sep = _2_sam
                
            
        elif (pres_abs == 0):
            desired_sep = torch.from_numpy(np.float32(np.zeros(_2_sam.shape[1])))
            desired_sep = torch.unsqueeze(desired_sep, 0)
        
        
        if self.transform: # May have to normalize (at the bottom) and resample and shit...
            desired_sep = self.transform(desired_sep) #16kz to 8kz
            mix_1_2 = self.transform(mix_1_2) #16kz to 8kz
            
        sample = {'conditional': cond_, 'sum_1_2': mix_1_2, 'separation': desired_sep, 'pres_abs':pres_abs}#, 'pres_abs_bin_flag': pres_abs_binaryFlag}

        return sample

In [ ]:

tr_d_ds = S2Lpairs_Dataset(csv_file = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/csv/tr_combinations_large.csv',
                     wav_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/split2sec/',
                     mix_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/processed_mixed_data/tr/',
                     transform = torchaudio.transforms.Resample(16000, 16000))

val_d_ds = S2Lpairs_Dataset(csv_file = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/csv/val_combinations.csv',
                     wav_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/val/',
                     mix_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/processed_mixed_data/val/',
                     transform = torchaudio.transforms.Resample(16000, 16000))

BATCH_SIZE = 24

# Setup the Dataloaders
tr_dataloader = DataLoader(tr_d_ds, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0) # there's also drop_last = False, collate_fn, etc.
val_dataloader = DataLoader(val_d_ds, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0) # there's also drop_last = False, collate_fn, etc.

for b_no,sample_batched in enumerate(val_dataloader):
    print("\n Conditional tensor:",sample_batched['conditional'].size(),
          "\n Mixture Tensor: ",sample_batched['sum_1_2'].size(),
          "\n Present/ Absent:",sample_batched['pres_abs'].size())
    
    print("\n -------------------------- Batch Number ----------------------------:",b_no)
    
    if(b_no == 1):
        break


 Conditional tensor: torch.Size([24, 1, 14]) 
 Mixture Tensor:  torch.Size([24, 1, 32000]) 
 Present/ Absent: torch.Size([24])

 -------------------------- Batch Number ----------------------------: 0

 Conditional tensor: torch.Size([24, 1, 14]) 
 Mixture Tensor:  torch.Size([24, 1, 32000]) 
 Present/ Absent: torch.Size([24])

 -------------------------- Batch Number ----------------------------: 1


In [ ]:
# Model definition based on the design of model #1

from asteroid_filterbanks import make_enc_dec
from asteroid_filterbanks import STFTFB

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        fb = STFTFB(n_filters=256, kernel_size=128, stride=64)
        self.enc = Encoder(fb)
        self.cond_enc = Encoder(fb)
     
        
        decoder_fb = STFTFB(n_filters=514, kernel_size=128, stride=64)
        self.dec = Decoder(decoder_fb)
        
        self.masker = TDConvNet(in_chan=516, 
                                n_src=1)
        self.m = nn.Linear(14, 64) 
        self.m2 = nn.Linear(64, 499)
        
        self.dropout = nn.Dropout(0.25)
        
        self.c1 = nn.Conv1d(in_channels=1, out_channels= 258, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        self.bn1 = nn.BatchNorm1d(258)
        
        self.c1d_1 = nn.Conv1d(in_channels=516, out_channels= 516, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        self.bn2 = nn.BatchNorm1d(516)
        
        
        # For the FC layers in the binary Classification
        self.co_fc_1 = nn.Conv1d(in_channels=516, out_channels= 1, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        
            
        # Number of input features is 516x499.
        self.fc1 = nn.Linear(499, 32) 
        self.fc2 = nn.Linear(32, 1)
        #self.fc3 = nn.Linear(32, 1) 
        
        #self.relu = nn.ReLU()
        self.dropout_2 = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        #self.softmax = nn.LogSoftmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, wav, cond_wav):
        
        #print("Shapes of the input mix and cond:")
        #print(wav.shape)
        #print(cond_wav.shape)
        tf_rep = self.enc(wav)
        #print("00_tf_rep",tf_rep.shape)

        m_out = self.m(cond_wav)
        m_out = F.relu(m_out) # need dropout
        m_out = self.dropout(m_out)
        #print("01_m_out", m_out.shape)
        
        m2_out = self.m2(m_out)
        m2_out = F.relu(m2_out) # need dropout, changed to sigmoid from relu
        m2_out = self.dropout(m2_out)
        #print("01_m2_out", m2_out.shape)
        
        c1_out = self.c1(m2_out)
        c1_out = F.relu(self.bn1(c1_out))
        #print("01_c1_out", c1_out.shape)
                
        concat_in = torch.cat((tf_rep, c1_out),dim=1)
        #print("03_concat_in:",concat_in.shape)
        
        c1d_1_out = self.c1d_1(concat_in)
        c1d_1_out = F.relu((c1d_1_out)) # removed <self.bn2> from here
        #print("04_c1d_1_out:",c1d_1_out.shape)

        masks = self.masker(c1d_1_out)
        #print("04_masks Shape: ",masks.shape)
        #print("Concat the above 04s")
        
        con_usq = c1d_1_out.unsqueeze(1) * masks
        #print("Post mask multiplication, con_usq:", con_usq.shape)
        
        # Add the pres_abs binary detector here
        co_fc_1_out = self.co_fc_1(con_usq.squeeze(1))
        #print("05_co_fc_1_out",co_fc_1_out.shape)
        
        x = co_fc_1_out.view(-1, 499)
        x = self.dropout_2(F.relu(self.fc1(x)))
        x = self.sigmoid(self.fc2(x)) # Pres/abs
        
        wavs_out = self.dec(con_usq) # output sep waveform
        
        return wavs_out, x

print(device)
# Define and forward 
cond_conv_tasnet = Model()
cond_conv_tasnet.to(device)
wav_out, pres_abs = cond_conv_tasnet(wav = torch.randn(10, 1, 32000).to(device), cond_wav = torch.randn(10, 1, 14).to(device))   
print(wav_out.shape, pres_abs.shape)

cuda
torch.Size([10, 1, 32000]) torch.Size([10, 1])


In [ ]:
def count_parameters(cond_conv_tasnet):
    return sum(p.numel() for p in cond_conv_tasnet.parameters() if p.requires_grad)

n = count_parameters(cond_conv_tasnet)
print("Number of parameters: %s" % n)

Number of parameters: 5822246


In [ ]:
# Set up loss funcion
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from='pw_mtx')

#optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the 
optimizer = optim.Adam(cond_conv_tasnet.parameters(), lr=1e-4)

In [ ]:
'''PATH = '../../../icassp2022/model123/Model2/saved_model/model2-5M_EP_last.pt'
PATH = './saved_model/ohot_m00-5M_EP_35.pt'


cond_conv_tasnet.load_state_dict(torch.load(PATH))'''

"PATH = '../../../icassp2022/model123/Model2/saved_model/model2-5M_EP_last.pt'\nPATH = './saved_model/ohot_m00-5M_EP_35.pt'\n\n\ncond_conv_tasnet.load_state_dict(torch.load(PATH))"

In [ ]:
import sklearn.metrics

def get_accuracy(y_true, y_prob):
    accuracy = sklearn.metrics.accuracy_score(y_true, y_prob > 0.5)
    return accuracy

In [ ]:
def train(model, epoch):
    model.train() # you need this to turn on the 'training' of certain aspects of the model like the BN and dropout
    # During evaluation/ inference, you should use model.eval()
    
    acc_acc = 0
    for batch_idx, sample_batched in enumerate(tr_dataloader):
        # data_1 = mixture | size 32000
        # data_2 = conditional | size 8
        
        
        sum_1_2 = sample_batched['sum_1_2'].to(device) # Mixture of two ROIs
        cond_lookUp = sample_batched['conditional'].to(device) # Conditional/What to Look for
        target = sample_batched['separation'].to(device) # Desired Separation
        x_target = sample_batched['pres_abs'].to(device).unsqueeze(1) # Fixing the Pres/abd binary array!
        x_target = x_target.to(torch.float32)
        
        output, x_out = model(wav = sum_1_2, cond_wav = cond_lookUp)

        loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from='pw_mtx')
        loss = loss_func(target, output)
        
        bce_loss_func = nn.BCELoss()
        try:
            bce_loss = bce_loss_func(x_out, x_target)
           
        except:
            print("Error occurs here, output:", x_out)
            print("Error occurs here, target:", x_target)
            print(">>> INPUT <<")
            print("Input Mixture:",sum_1_2)
            print("Conditional",cond_lookUp)
            print("Detection Target",x_target)
            print("Separation Desired:",target)
            
            joblib.dump(sum_1_2,'./problem_inputs/dual_sum_1_2.pkl')
            joblib.dump(cond_lookUp, './problem_inputs/dual_cond_lookUp.pkl')
            joblib.dump(x_target, './problem_inputs/dual_x_target.pkl')
            joblib.dump(target, "./problem_inputs/dual_sep_target.pkl")
            
        loss = loss + bce_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = get_accuracy(torch.squeeze(x_target,1).cpu().data.numpy(), torch.squeeze(x_out,1).cpu().data.numpy())
        acc_acc += acc
            
        # print training stats
        if batch_idx % 10000 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(sum_1_2)}/{len(tr_dataloader.dataset)} ({100. * batch_idx / len(tr_dataloader):.0f}%)]\tLoss: {loss.item():.6f}")
            print("Batch_idx:",batch_idx,"|| Accuracy:",acc,"||AccAcc:",acc_acc/(batch_idx+1))
        
        losses.append(loss.item())

In [ ]:
def test(model):
    
    cond_conv_tasnet.eval()
    test_loss = 0
    correct = 0
    best_loss = 1
    acc_acc = 0
    
    example_images = []
    
    with torch.no_grad():
        for batch_idx, sample_batched in enumerate(val_dataloader):

            sum_1_2 = sample_batched['sum_1_2'].to(device) # Mixture of two ROIs
            cond_lookUp = sample_batched['conditional'].to(device) # Conditional/What to Look for
            target = sample_batched['separation'].to(device) # Desired Separation
            x_target = sample_batched['pres_abs'].to(device).unsqueeze(1) # Fixing the Pres/abd binary array!
            x_target = x_target.to(torch.float32)

            output, x_out = model(wav = sum_1_2, cond_wav = cond_lookUp)
            
            acc = get_accuracy(torch.squeeze(x_target,1).cpu().data.numpy(), torch.squeeze(x_out,1).cpu().data.numpy())
            acc_acc += acc
            
            if batch_idx % 10 == 0:
                print("Batch_idx:",batch_idx,"|| Accuracy:",acc,"||AccAcc:",acc_acc/(batch_idx+1))
                #print(f"Train: {batch_idx} [{batch_idx * len(cond_lookUp)}/{len(val_dataloader.dataset)} ({100. * batch_idx / len(val_dataloader):.0f}%)]\tLoss: {test_loss.item():.6f}")



In [12]:
n_epoch = 50
losses = []

for epoch in range(1, n_epoch + 1):
    print('train:')
    train(cond_conv_tasnet, epoch)
    print('test:')
    test(cond_conv_tasnet)
    print('-'*100)
    
    PATH = './saved_model/ohot_Nigens-5M_EP_'+str(epoch)+'.pt'
    torch.save(cond_conv_tasnet.state_dict(), PATH)

train:
Train Epoch: 1 [0/551359 (0%)]	Loss: 62.574856
Batch_idx: 0 || Accuracy: 0.3333333333333333 ||AccAcc: 0.3333333333333333
Train Epoch: 1 [240000/551359 (44%)]	Loss: 48.251163
Batch_idx: 10000 || Accuracy: 0.6666666666666666 ||AccAcc: 0.6905976069059733
Train Epoch: 1 [480000/551359 (87%)]	Loss: 46.924068
Batch_idx: 20000 || Accuracy: 0.7916666666666666 ||AccAcc: 0.696640167991599
test:
Batch_idx: 0 || Accuracy: 0.0 ||AccAcc: 0.0
Batch_idx: 10 || Accuracy: 0.5 ||AccAcc: 0.4659090909090909
Batch_idx: 20 || Accuracy: 0.2916666666666667 ||AccAcc: 0.46626984126984117
Batch_idx: 30 || Accuracy: 0.4583333333333333 ||AccAcc: 0.49059139784946226
Batch_idx: 40 || Accuracy: 0.45 ||AccAcc: 0.506910569105691
----------------------------------------------------------------------------------------------------
train:
Train Epoch: 2 [0/551359 (0%)]	Loss: 38.753296
Batch_idx: 0 || Accuracy: 0.6666666666666666 ||AccAcc: 0.6666666666666666
Train Epoch: 2 [240000/551359 (44%)]	Loss: 34.303852
Batch_i

KeyboardInterrupt: 

In [ ]:
stoppp

In [ ]:
PATH = './saved_model/00_wav_cond_03.pt'
torch.save(cond_conv_tasnet.state_dict(), PATH)

In [ ]:
test(cond_conv_tasnet)

In [ ]:
# Inference:

spec = ['alarm','baby','crash','dog','engine','femaleScream',
                 'femaleSpeech','fire','footsteps','knock','maleScream',
                 'maleSpeech','phone','piano']

val_dataloader = DataLoader(val_d_ds, batch_size = 150, shuffle = True, num_workers = 1) # there's also drop_last = False, collate_fn, etc.

device = 'cuda'

for b_no,sample_batched in enumerate(val_dataloader):
    print(sample_batched['conditional'].size(),
          sample_batched['sum_1_2'].size(),
          sample_batched['separation'].size(),
          sample_batched['pres_abs'].size())
    
    sum_1_2 = sample_batched['sum_1_2'].to(device) # Mixture of two ROIs
    cond_lookUp = sample_batched['conditional'].to(device) # Conditional/What to Look for
    target = sample_batched['separation'].to(device) # Desired Separation
    x_target = sample_batched['pres_abs'].to(device).unsqueeze(1) # Fixing the Pres/abd binary array!
    x_target = x_target.to(torch.float32)

    
    print(b_no)
    
    if(b_no == 5):
        break
        


In [ ]:
cond_conv_tasnet.eval()
with torch.no_grad():
    output, x_out = cond_conv_tasnet(wav = sum_1_2, cond_wav = cond_lookUp)

import librosa.display

def showSpec(y = None):
    S = librosa.feature.melspectrogram(y=y.cpu().data.numpy().flatten(), sr=16000, n_mels=64,fmin = 0,
                         fmax=8000, n_fft=728, hop_length=32, win_length = None, htk = True) 
    librosa.display.specshow(librosa.power_to_db(S ** 1, ref=np.max), fmin=0, y_axis='linear')
    plt.show()


In [ ]:
no = np.random.randint(len(target))
print(no)

print("Pred:",x_out[no].cpu().data)
print("GroundTruth:",x_target[no].cpu().data)
#print("LOOK FOR --",spec[np.argmax(cond_lookUp[no].cpu().data)])
print('-'*100)

print("Conditional Truth:")
print(cond_lookUp[no].cpu().data.numpy().flatten())
spec = ['alarm','baby','crash','dog','engine','femaleScream',
                 'femaleSpeech','fire','footsteps','knock','maleScream',
                 'maleSpeech','phone','piano']
argMx = np.argmax(cond_lookUp[no].cpu().data.numpy().flatten())
print(">> Cond - ", spec[argMx])


print("Predicted:")
#output[no] # Synthesized separation
plt.plot(output[no].cpu().data.numpy().flatten()) 
plt.show()
showSpec(y=output[no])
ipd.display(ipd.Audio(output[no].cpu().data.numpy().flatten(), rate = 16000))


print("Ground Truth:")
target[no] # true separation
plt.plot(target[no].cpu().data.numpy().flatten())
plt.show()
showSpec(y=target[no])
ipd.display(ipd.Audio(target[no].cpu().data.numpy().flatten(), rate = 16000))
'''
x_out[no] # pres/abs 1/0 flag
print(x_out[no])

x_target[no] # true pres/abs detector
print(x_target[no])

'''
print("MIXTURE:")
sum_1_2[no] # Input Mixture
plt.plot(sum_1_2[no].cpu().data.numpy().flatten())
plt.show()
showSpec(y=sum_1_2[no])
ipd.display(ipd.Audio(sum_1_2[no].cpu().data.numpy().flatten(), rate = 16000))

In [ ]:
import numpy as np
import pprint
from asteroid.metrics import get_metrics

'''metrics_dict = get_metrics(pt_mix_sam, pt_c1_sam, output, sample_rate=8000,
                            metrics_list='all')'''
mix = sum_1_2[no].cpu().data.numpy()
clean = target[no].cpu().data.numpy()
est = output[no].cpu().data.numpy()

metrics_dict = get_metrics(mix, clean, est, sample_rate=8000,
                            metrics_list='all')

pprint.pprint(metrics_dict)

In [ ]:
true_pres_abs = x_target[no].cpu().data
print("true_pres_abs:",true_pres_abs.shape)

pred_pres_abs = x_out[no].cpu().data
print("pred_pres_abs:",pred_pres_abs.shape)

true_mix = sum_1_2[no].cpu().data.numpy().flatten()
print("true_mix shape:", true_mix.shape)aa

look_for = cond_lookUp[no].cpu().data.numpy().flatten()
print("look_for:",look_for.shape)

true_sep = target[no].cpu().data.numpy().flatten()
print("true_sep:", true_sep.shape)

pred_sep = output[no].cpu().data.numpy().flatten()
print("pred_sep:",pred_sep.shape)

In [ ]:
import soundfile as sf
import joblib

save_here = os.path.join('./saved_sams/wav_condModel03/','good_cond_wav'+'_'+str(np.random.randint(100000)))
#save_here = os.path.join('./saved_sams/wav_condModel03/','supress_bad__cond_wav'+'_'+str(np.random.randint(100000)))
print(save_here)
os.mkdir(save_here)
joblib.dump(true_pres_abs, os.path.join(save_here,'true_pres_abs.pkl'))  
joblib.dump(pred_pres_abs, os.path.join(save_here, 'pred_pres_abs.pkl'))
joblib.dump(true_mix, os.path.join(save_here, 'true_mix.pkl'))
joblib.dump(look_for, os.path.join(save_here, 'look_for.pkl'))
joblib.dump(true_sep, os.path.join(save_here, 'true_sep.pkl'))
joblib.dump(pred_sep, os.path.join(save_here, 'pred_sep.pkl'))